# Demande immobilière en France

Par Yanis Larré, Tom Croquette et Renan Romariz.


## Introduction 

Nous nous proposons ici d'étudier la demande immobilière en France. Mettre de la biblio + idées + NB


## Sommaire

* [Installation](#installation)


## I. Installation
La première étape du projet consiste à télécharger tous les modules et fonctions dont nous auront besoin.

In [46]:
!pip install -r requirements.txt
!pip install openpyxl

# Modules :
import os
import s3fs
import pandas as pd

# Fonctions :
from scripts import get_data
from scripts import data_clean
from scripts import data_analysis
from scripts import data_visualization
from scripts import do_ols

In [33]:
%load_ext autoreload
%autoreload 2


## II. Préparation des données

Nous téléchargeons dans cette parties les données utiles dans notre projet. Nous procédons également au nettoyage de celles-ci afin de pouvoir les exploiter convenablement.

### 1. Données immobilières

Nous décidons de récupérer des données foncières sur le site *data.gouv*, en utilisant plus particulièrement les données DVF (Demande de Valeur Foncière).
Ces données concernent la quasi-totalité de la France métropolitaine et des département d'Outre-Mer. En effet, les données de Mayotte ne sont pas incluses, ainsi que celles de 3 départements métropolitains : le Bas-Rhin (67), le Haut-Rhin (68) et la Moselle (57), car ces département utilisent un système hérité de l'annexion allemande appelé le Livre Foncier. Leurs données ne sont pas centralisées par la DGFiP de la même manière, donc elles ne redescendent pas dans la base nationale DVF.


In [22]:
df = get_data.get_dvf()

/home/onyxia/work/Python_2A/scripts/get_data.py:23: DtypeWarning: Columns (12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype={'code_commune': "str"})


#### Nettoyage

Dans notre analyse, nous avons pris le parti de ne considérer que les maisons et appartements. En effet, la base de données contient d'autres catégories de biens immobiliers telles que les terrains vierges par exemple. Toutefois, les maisons et appartements sont les biens susceptibles de montrer les résultats les plus parlants ; ce sont pas ailleurs les catégories proposant les données le plus complètes sur le prix, la localisation et la surface notamment.

In [23]:
df = df[df['type_local'].isin(['Maison', 'Appartement'])]
df = df.reset_index()

Ensuite, les codes communes n'ont pas tous le même type : certains sont de type *int*, d'autres de type *string*. On convertit l'ensemble au type *string* car certains codes communes (notamment ceux des communes corses, commençant par 2A ou 2B) ne peuvent pas être convertis en type *int*.

In [24]:
data_clean.convertir_codes_communes(df)
print(df['code_commune'])

0          01130
1          01451
2          01364
3          01053
4          01177
           ...  
6247210    75111
6247211    75112
6247212    75116
6247213    75113
6247214    75105
Name: code_commune, Length: 6247215, dtype: object


Enfin, certaines ventes sont réalisées par lots : on décide simplement des les supprimer du dataset. En effet, on ne peut pas retrouver la valeur foncière des éléments individuels, et on souhaite aussi se concentrer sur les logements anciens (marché de l'occasion, plutôt que les ventes de maisons et appartements neufs souvent vendus en lots par les promoteurs).

In [38]:
comptage_df = df['id_mutation'].value_counts().reset_index()
comptage_df.columns = ['id_mutation', 'nombre_de_lignes']
print(comptage_df)

id_uniques = comptage_df[comptage_df['nombre_de_lignes'] == 1]
print(id_uniques.shape)

print(id_uniques)
df_sans_lots=df[df['id_mutation'].isin(id_uniques['id_mutation'])]
print(df_sans_lots)

         id_mutation  nombre_de_lignes
0        2021-687176              9290
1        2022-919567              7657
2        2022-909252              7477
3        2021-342324              7112
4        2020-491811              6173
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4861637 rows x 2 columns]
(4133749, 2)
         id_mutation  nombre_de_lignes
727888   2023-581082                 1
727889   2023-510332                 1
727890   2023-580244                 1
727891   2023-510334                 1
727892   2023-579686                 1
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4133749 rows x 2 colu

#### Aperçus des données

En calculant le nombre de ventes par commune, on remarque que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025 (50684 ventes).

In [25]:
ventes_par_commune = df['code_commune'].value_counts().reset_index(name='nombre')

# Premiers résultats :
print(ventes_par_commune)
print( ventes_par_commune["nombre"][ventes_par_commune["code_commune"]=='2A004'])
print(ventes_par_commune['nombre'].max())
print(ventes_par_commune['nombre'].idxmax())
print(ventes_par_commune['code_commune'][ventes_par_commune['nombre'].idxmax()])
print(ventes_par_commune['nombre'][ventes_par_commune['nombre'].idxmax()])

      code_commune  nombre
0            31555   50684
1            06088   46697
2            44109   30331
3            33063   30077
4            34172   28927
...            ...     ...
33265        52549       1
33266        21136       1
33267        21653       1
33268        52053       1
33269        52505       1

[33270 rows x 2 columns]
145    4880
Name: nombre, dtype: int64
50684
0
31555
50684


### 2. Données de population

Afin d'étudier la dynamique autour du secteur immobilier (restreint aux maisons et aux appartements), il nous est nécessaire d'utiliser les données de population du territoire français. Pour cela, nous utilisons les données de XXX.

In [26]:
df_pop = get_data.get_pop()

Certaines valeurs ne sont toutefois pas renseignées ; on remarque aussi que certaines communes ont des populations très faibles.
Pour que le ratio des ventes sur la population ait du sens, il nous faut éliminer les communes ayant une population très faible (en particulier, la commune de Bezonvaux ne comptant aucun habitant...).
On imposte donc un nombre minimum d'habitants arbitraire, et on exclut les communes de population trop faible ou non renseignée.

PARTIE INUTILE MAISEFFECTUE PLUS TARD ASKIP

In [27]:
lignes_na = data_clean.filtre_donnes_pop(df_pop)
print(lignes_na)

       objectid  reg dep    cv code_commune                  libgeo  p13_pop  \
3435     128717   26  21  2115        21507               Premières    140.0   
8276     116240   24  45  4504        45287    Saint-Loup-de-Gonois     97.0   
11971    143352   52  53  5306        53274                 Vimarcé    236.0   
12193    143465   54  16  1607        16351            Saint-Simeux    589.0   
19186    134748   52  53  5306        53239  Saint-Martin-de-Connée    411.0   
26133    122832   23  27  2712        27676                Venables    794.0   
28348    125987   26  21  2107        21213               Crimolois    759.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
3435     140.0    141.0    141.0      141      NaN      NaN      NaN      NaN  
8276      91.0     89.0     86.0       97      NaN      NaN      NaN      NaN  
11971    242.0    241.0    240.0      237    234.0      NaN      NaN      NaN  
12193    598.0    607.0    610.0      5

Nous remarquons que certains codes communes sont à six chiffres. En effet, dans les DOMs, le numéro commence par un code département à trois chiffres plutôt qu'à deux (pour différencier les DOMs).
Cela n'est pas un problème en soit, mais la convention adoptée dans le fichier DVF est différente ; on ne rajoute pas le troisième chiffre.
Par exemple, le code commune est différent dans *df_pop* et *df* pour la commune de Salazie, à la Réunion (974421 dans *df_pop*, 97421 dans *df*) : la Réunion est perçue comme le département 974 dans *df_pop*. On va ainsi supprimer le troisième chiffre de chacun des codes communes à 6 chiffres.


In [28]:
# Problème des codes communes à 6 chiffres :
print(df_pop[df_pop['code_commune']=='974421'])
lignes_6_chiffres = df_pop[df_pop["code_commune"].str.len() == 6]
print(lignes_6_chiffres)
print(df_pop[df_pop['code_commune']=='974421']['libgeo'])
print(df[df['code_commune']=='97421']['nom_commune'])

      objectid  reg  dep     cv code_commune   libgeo  p13_pop  p14_pop  \
3786    131030    4  974  97406       974421  Salazie   7226.0   7132.0   

      p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
3786   7384.0   7400.0     7312   7224.0   7136.0   7310.0   7243.0  
       objectid  reg  dep     cv code_commune                  libgeo  \
84       126394    3  973    973       973353             Maripasoula   
192      126587    3  973    973       973313  Montsinéry-Tonnegrande   
206      126601    3  973    973       973358              Saint-Élie   
210      126605    1  971  97114       971102           Anse-Bertrand   
748      130678    1  971  97195       971106              Bouillante   
...         ...  ...  ...    ...          ...                     ...   
33141    115576    2  972    972       972225            Saint-Pierre   
33257    111336    4  974  97401       974404           L' Étang-Salé   
33557    111383    2  972    972       972201      L

In [32]:
# Harmonisation :
df_pop["code_commune"] = df_pop["code_commune"].apply(data_clean.enleverchiffreDOMs)
print(df_pop)
lignes_6_chiffres = df_pop[df_pop["code_commune"].str.len() == 6]
print(lignes_6_chiffres)
print(df_pop[df_pop['code_commune']=='974421'])

       objectid  reg dep    cv code_commune                 libgeo  p13_pop  \
0        115658   52  85  8502        85062            Châteauneuf    968.0   
1        115659   26  58  5808        58300                   Urzy   1839.0   
2        115660   43  70  7012        70137  Chassey-lès-Montbozon    218.0   
3        115661   21  51  5123        51649      Vitry-le-François  13174.0   
4        115662   11  78  7811        78638         Vaux-sur-Seine   4749.0   
...         ...  ...  ..   ...          ...                    ...      ...   
34990    110251   31  62  6225        62327         Febvin-Palfart    574.0   
34991    110252   73  12  1201        12156            Montpeyroux    550.0   
34992    110253   54  86  8610        86138                Luchapt    275.0   
34993    110254   72  33  3320        33184                Générac    579.0   
34994    110255   72  24  2409        24354       La Roche-Chalais   2932.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop 

#### Ventes par commune
On joint ensuite les données de population à celles des données foncières.

In [37]:
ventes_par_commune_par_habitant = pd.merge(ventes_par_commune, df_pop, on='code_commune')
ventes_par_commune_par_habitant = ventes_par_commune_par_habitant[
    (ventes_par_commune_par_habitant['p19_pop'].notna()) &
    (ventes_par_commune_par_habitant['p19_pop'] > 1000)
]
ventes_par_commune_par_habitant['ventes par habitants par commune'] = ventes_par_commune_par_habitant['nombre']/ventes_par_commune_par_habitant['p19_pop']

print(ventes_par_commune_par_habitant['ventes par habitants par commune'])
print(ventes_par_commune_par_habitant['ventes par habitants par commune'].max())
print(ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax())
print(ventes_par_commune_par_habitant['code_commune'][ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax()])

0        0.102710
1        0.136274
2        0.095139
3        0.115256
4        0.097878
           ...   
22363    0.014115
26593    0.015238
30690    0.005638
31746    0.000683
32063    0.000527
Name: ventes par habitants par commune, Length: 9422, dtype: float64
1.172281776416539
611
38191


#### Tentative de récupération des prix au mètre carré

Remarque : on ne considère que le ratio *valeur foncière/surface batîe* de la maison ou de l'appartement, puisqu'on ignore les dépendances ou autres terrains additionnels.
On sur-estime donc la valeur au m2. On ignore les lots (on utilise *df_sans_lots* qui ne contient pas les observations dont l'id_mutation apparaît plusieurs fois).


In [40]:
df_sans_lots = df_sans_lots.copy()
print(df_sans_lots.shape)

df_sans_lots = df_sans_lots[
    (df_sans_lots['surface_reelle_bati'].notna()) &
    (df_sans_lots['valeur_fonciere'].notna()) &
    (df_sans_lots['surface_reelle_bati'] > 10)
]

# Définition du rapport de la valeur foncière sur la surface :
df_sans_lots['rapport valeur foncière et surface bâtie']=df_sans_lots['valeur_fonciere']/df_sans_lots['surface_reelle_bati']

print(df_sans_lots['rapport valeur foncière et surface bâtie'])
print(df_sans_lots['rapport valeur foncière et surface bâtie'].max())
print(df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax())
print(df_sans_lots.loc[df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax()])

(4133749, 41)
0            669.565217
1           2286.885246
2           2220.625000
3           1966.406977
4           1666.666667
               ...     
6247210    11928.571429
6247211    11363.636364
6247212    14891.304348
6247213     9016.393443
6247214    11710.342553
Name: rapport valeur foncière et surface bâtie, Length: 4125604, dtype: float64
15937500.0
5816283
index                                                      18656556
id_mutation                                            2024-1198671
date_mutation                                            2024-06-27
numero_disposition                                                1
nature_mutation                                               Vente
valeur_fonciere                                         255000000.0
adresse_numero                                                 42.0
adresse_suffixe                                                 NaN
adresse_nom_voie                                       AV MONTAIGNE
adresse_cod

Problème : on trouve des valeurs anormalement élevées pour certains logements, correspondant probablement à des immeubles entiers plutôt qu'à des logements (ci-dessus, immeuble en construction de presque 1 milliard d'euros...).

In [42]:
# Moyenne par commune :
moyenne_par_commune=df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()
print(moyenne_par_commune['rapport valeur foncière et surface bâtie'].max())
print(moyenne_par_commune['code_commune'].loc[moyenne_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])


# Médiane par commune
mediane_par_commune = df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()
# Valeur maximale de la médiane
print(mediane_par_commune['rapport valeur foncière et surface bâtie'].max())
# Code de la commune avec la médiane maximale
print(mediane_par_commune['code_commune'][mediane_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])

39178.54209868362
25448
14752.04918032787
75106


Meilleure alternative au troncage précédent [QUEL TRONCAGE PRECEDENT ? --> préciser et ajouter le code] : tronquer par commune. On trouve alors des médianes et des moyennes très proches, maximales pour le sixième arrondissement de Paris, ce qui est cohérent !


In [44]:
df_sans_lots_tronqué = data_clean.troncature_lots(df_sans_lots)


# Calcul de la moyenne par commune (tronqué)
moyenne_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

# Affichage de la moyenne maximale et de la commune associée
print(moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(moyenne_par_commune_tronqué['code_commune'].loc[moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])

# Calcul de la médiane par commune (tronqué)
mediane_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Affichage de la médiane maximale et de la commune associée
print(mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(mediane_par_commune_tronqué['code_commune'][mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])

14889.448668014387
75106
14752.04918032787
75106


On crée, enfin, un dataframe complet contenant toutes les données pertinentes par commune.
Attention : les données sur les ventes par habitant ont été crées plus tôt en se restreignant aux communes avec au moins un certain nombre d'ventes_par_commune_par_habitant_simple.
#Remonter et changer de seuil si on veut élargir au restreindre le champ des données
#!!!Dans tous les cas il y aura pas mal de valeurs manquantes pour les données en question, bien faire gaffe à ce qu'on étudie!!!


In [47]:
pop_par_commune=df_pop[["code_commune", "p19_pop","libgeo"]]
ventes_par_commune_par_habitant_simple = ventes_par_commune_par_habitant[["code_commune", "ventes par habitants par commune"]]


df_final=pd.merge(pop_par_commune, ventes_par_commune, how='outer', on="code_commune")
df_final.rename(columns={'nombre': 'nombre de ventes dans la commune'}, inplace=True)
df_final.rename(columns={'libgeo': 'nom de la commune'}, inplace=True)
df_final.rename(columns={'p19_pop': 'population en 2019'}, inplace=True)
df_final=pd.merge(df_final, ventes_par_commune_par_habitant_simple, how='outer', on="code_commune")
df_final.rename(columns={'ventes par habitants par commune': 'nombre de ventes par habitant en 2019 dans la commune'}, inplace=True)
df_final=pd.merge(df_final, mediane_par_commune_tronqué , how='outer', on="code_commune")
df_final.rename(columns={'rapport valeur foncière et surface bâtie': 'médiane tronquée du prix au m2'}, inplace=True)
df_final=pd.merge(df_final, moyenne_par_commune_tronqué , how='outer', on="code_commune")
df_final.rename(columns={'rapport valeur foncière et surface bâtie': 'moyenne tronquée du prix au m2'}, inplace=True)
print(df_final)

print(df_final['moyenne tronquée du prix au m2'].max())
print(df_final['code_commune'][df_final['moyenne tronquée du prix au m2'].idxmax()])

      code_commune  population en 2019         nom de la commune  \
0            01001               779.0  L' Abergement-Clémenciat   
1            01002               256.0    L' Abergement-de-Varey   
2            01004             14134.0         Ambérieu-en-Bugey   
3            01005              1751.0       Ambérieux-en-Dombes   
4            01006               112.0                   Ambléon   
...            ...                 ...                       ...   
34995        97420             24065.0            Sainte-Suzanne   
34996        97421              7136.0                   Salazie   
34997        97422             79824.0                 Le Tampon   
34998        97423              7015.0         Les Trois-Bassins   
34999        97424              5538.0                    Cilaos   

       nombre de ventes dans la commune  \
0                                  59.0   
1                                  21.0   
2                                1280.0   
3          

Il reste des NaN dans le df_final... ????????????

In [48]:
df_final

,code_commune,population en 2019,nom de la commune,nombre de ventes dans la commune,nombre de ventes par habitant en 2019 dans la commune,médiane tronquée du prix au m2,moyenne tronquée du prix au m2
0,01001,779.0,L' Abergement-Clémenciat,59.0,NaN,2431.034483,2483.602665
1,01002,256.0,L' Abergement-de-Varey,21.0,NaN,1769.026549,1719.646511
2,01004,14134.0,Ambérieu-en-Bugey,1280.0,0.090562,2284.701114,2354.881148
3,01005,1751.0,Ambérieux-en-Dombes,235.0,0.134209,2812.820513,2803.968537
4,01006,112.0,Ambléon,16.0,NaN,1688.688525,1626.384067
...,...,...,...,...,...,...,...
34995,97420,24065.0,Sainte-Suzanne,1042.0,0.043299,2169.996032,2198.182140
34996,97421,7136.0,Salazie,228.0,0.031951,1458.185684,1396.345521
34997,97422,79824.0,Le Tampon,4277.0,0.053580,2228.571429,2393.514030
34998,97423,7015.0,Les Trois-Bassins,258.0,0.036778,2500.000000,3017.406091


## III. Analyse

### 1. Données de régression

Nous allons tenter d'effectuer une régression du nombre de ventes par communes sur différents paramètres. Nous choisissons en premier lieu trois paramètres variés : l'IDH (Indice de Développement Humain), la criminalité et la population de pharmacie.

In [50]:
pharmacies = get_data.get_local_csv('pharmacies_point')
idh = get_data.get_local_csv('indice-de-developpement-humain-idh2-des-communes-dile-de-france', sep=';')
crime = get_data.get_local_csv('donnee-dep-data.gouv-2024-geographie2025-produit-le2025-06-04', sep=';')

/home/onyxia/work/Python_2A/scripts/get_data.py:52: DtypeWarning: Columns (4,11,16,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, sep=sep)


## Conclusion et perspectives

En conclusion, 